# An example of PBC-SCF implementation
## https://github.com/sunqm/pbchf

In [1]:
import numpy as np
from pyscf.pbc import gto

TEST = 1

In [2]:
cell = gto.Cell()
cell.a = np.eye(3) * 4.
cell.atom = 'He 2 2 2'
cell.basis = 'unc-sto-3g'
cell.unit = 'Bohr'
cell.build()

## Gamma Point
### Grids for numerical integration

In [3]:
def get_grids(a, mesh):
    grids_x = a[0,0] * np.arange(mesh[0]) / mesh[0]
    grids_y = a[1,1] * np.arange(mesh[1]) / mesh[1]
    grids_z = a[2,2] * np.arange(mesh[2]) / mesh[2]

    grids = []
    for x in grids_x:
        for y in grids_y:
            for z in grids_z:
                grids.append((x, y, z))
    return np.array(grids)

def get_grids_ref(a, mesh):
    grids_x = a[0,0] * np.arange(mesh[0]) / mesh[0]
    grids_y = a[1,1] * np.arange(mesh[1]) / mesh[1]
    grids_z = a[2,2] * np.arange(mesh[2]) / mesh[2]

    grids = np.empty([mesh[0], mesh[1], mesh[2], 3])
    grids[:,:,:,0] = grids_x[:,None,None]
    grids[:,:,:,1] = grids_y[None,:,None]
    grids[:,:,:,2] = grids_z[None,None,:]
    return grids.reshape(-1, 3)

if TEST:
    mesh = [20,20,20]
    print(abs(get_grids(cell.a, mesh) -
              get_grids_ref(cell.a, mesh)).max())

get_grids = get_grids_ref

### The value of PBC basis functions on real space grids

\begin{equation}
\phi(\mathbf{r}) = \frac{1}{\sqrt{N}}\sum_{\mathbf{L}} \mu(\mathbf{r}-\mathbf{L})
\end{equation}

In [4]:
def get_ao_values(cell, n_images, mesh):
    cell = cell.copy()
    grids = get_grids(cell.a, mesh)
    Ls = get_lattice_Ls(cell, n_images)
    atom_ref = cell.atom_coords()

    ao = 0
    for L in Ls:
        cell.atom = [['He', atom_ref[0] + L]]
        cell.build()
        ao += cell.eval_gto('GTOval', grids)
    return ao

def get_lattice_Ls(cell, n_images):
    n_images_x = n_images[0]
    n_images_y = n_images[1]
    n_images_z = n_images[2]
    Ls = []
    for ix in range(-n_images_x, n_images_x+1):
        for iy in range(-n_images_y, n_images_y+1):
            for iz in range(-n_images_z, n_images_z+1):
                L = np.einsum('x,xy->y', (ix, iy, iz), cell.a)
                Ls.append(L)
    return Ls

def get_ao_values_ref(cell, mesh=None):
    if mesh is None:
        mesh = cell.mesh
    grids = get_grids(cell.a, mesh)
    return cell.pbc_eval_gto('GTOval', grids)

if TEST:    
    mesh = [10,10,10]
    ao_ref = get_ao_values_ref(cell, mesh)
    ao = get_ao_values(cell, [0,0,0], mesh)
    print(abs(ao-ao_ref).max())
    ao = get_ao_values(cell, [1,1,1], mesh)
    print(abs(ao-ao_ref).max())
    ao = get_ao_values(cell, [2,2,2], mesh)
    print(abs(ao-ao_ref).max())

get_ao_values = get_ao_values_ref

### Overlap integrals
The first method
\begin{equation}
\langle \phi_{\mu} | \phi_{\nu} \rangle
= \frac{1}{N} \int \phi_\mu(\mathbf{r})^* \phi_\nu(\mathbf{r}) d\mathbf{r}
= \sum_{\mathbf{r}\in \Omega} w \phi_\mu(\mathbf{r})^* \phi_\nu(\mathbf{r})
\end{equation}

In [5]:
def get_weight(cell):
    return cell.vol / np.prod(cell.mesh)

def get_ovlp1(cell):
    w = get_weight(cell)
    ao = get_ao_values(cell)
    s = w * np.einsum('xi,xj->ij', ao.conj(), ao)
    return s

The second method
\begin{align}
\langle \phi_\mu | \phi_\nu \rangle
&= \frac{1}{N}\sum_\mathbf{LL'}\int \mu(\mathbf{r}-\mathbf{L})^* \nu(\mathbf{r} - \mathbf{L}') d\mathbf{r} \\
&= \frac{1}{N}\sum_\mathbf{LL'}\int \mu(\mathbf{r})^* \nu(\mathbf{r} + \mathbf{L} - \mathbf{L}') d\mathbf{r} \\
&= \frac{1}{N}\sum_\mathbf{L} \sum_\mathbf{L'}\int \mu(\mathbf{r})^* \nu(\mathbf{r} + \mathbf{L'}) d\mathbf{r} \\
&= \int_{-\infty}^{\infty} \mu(\mathbf{r})^* \nu(\mathbf{r} + \mathbf{L'}) d\mathbf{r}
\end{align}

In [6]:
def get_ovlp2(cell):
    from pyscf import gto as mole_gto
    cellL = cell.copy()
    grids = get_grids(cell.a, cell.mesh)
    Ls = get_lattice_Ls(cell, cell.nimgs)
    atom_ref = cell.atom_coords()

    s = 0
    for L in Ls:
        cellL.atom = [['He', atom_ref[0] + L]]
        cellL.build()
        s += mole_gto.intor_cross('int1e_ovlp', cell, cellL)
    return s

def get_ovlp_ref(cell):
    return cell.pbc_intor('int1e_ovlp')
get_ovlp = get_ovlp_ref

if TEST:
    print(abs(get_ovlp1(cell) - get_ovlp2(cell)).max())
    print(abs(get_ovlp1(cell) - get_ovlp_ref(cell)).max())

The third method to compute overlap integrals:
 
\begin{equation}
\phi(\mathbf{r}) \rightarrow \phi(\mathbf{G})
\end{equation}
where $\mathbf{G}$ is the plane-wave vector ("Gv" in the code).
Gv can be obtained by Fourier transform
\begin{equation}
\phi(\mathbf{G}) = \int_\Omega e^{-i\mathbf{G}\cdot\mathbf{r}} \phi(\mathbf{r}) d\mathbf{r}
\end{equation}

The discrete FT sample frequencys $G_x$ are
\begin{gather}
G_x = \frac{2\pi n}{N L_x} \\
\begin{cases}
n = 0,\dots,N/2-1,\, -N/2,\dots,1 & N \text{ is even} \\
n = 0,\dots,(N-1)/2,\, -(N-1)/2,\dots,1 & N \text{ is odd}
\end{cases}
\end{gather}

In [7]:
def get_Gv(cell):
    # Planewaves that corresponds to the mesh in real space
    mesh = cell.mesh
    Gx = np.fft.fftfreq(mesh[0], 1./mesh[0])
    Gy = np.fft.fftfreq(mesh[1], 1./mesh[1])
    Gz = np.fft.fftfreq(mesh[2], 1./mesh[2])

    Gv = np.empty([mesh[0], mesh[1], mesh[2], 3])
    Gv[:,:,:,0] = Gx[:,None,None]
    Gv[:,:,:,1] = Gy[None,:,None]
    Gv[:,:,:,2] = Gz[None,None,:]

    b = cell.reciprocal_vectors()
    Gv = np.dot(Gv, b)
    return Gv

In [8]:
def get_aoG_values_ref(cell):
    from pyscf.pbc import df
    aoG = df.ft_ao.ft_ao(cell, cell.Gv)
    return aoG

from pyscf.pbc.tools import fft, ifft
def get_aoG_values(cell):
    aoR = get_ao_values(cell)
    nao = aoR.shape[1]
    aoG = []
    for i in range(nao):
        aoG.append(_______ * fft(aoR[:,i], cell.mesh))
    aoG = np.array(aoG).T
    return aoG

if TEST:
    print abs(get_aoG_values(cell) - get_aoG_values_ref(cell)).max()

get_aoG_values = get_aoG_values_ref

The overlap integrals can be numerically evaluated in reciprocal space
\begin{align}
\langle \phi_\mu | \phi_\nu \rangle
&= \int \phi_\mu(\mathbf{G})^* \phi_nu(\mathbf{G}) d\mathbf{G} \\
&= \sum_{\mathbf{G}} w_\mathbf{G} \phi_\mu(\mathbf{G})^* \phi_nu(\mathbf{G})
\end{align}

In [9]:
get_aoG_values = get_aoG_values_ref
def get_ovlp3(cell):
    aoR = get_ao_values(cell)
    nao = aoR.shape[1]
    aoG = get_aoG_values(cell)
    w = 1./cell.vol
    s = w * np.einsum('xi,xj->ij', aoG.conj(), aoG)
    return s

if TEST:
    print(abs(get_ovlp3(cell) - get_ovlp_ref(cell)).max())

### Kinetic integrals
Method 1:
\begin{align}
\langle \phi_\mu | -\frac{1}{2}\nabla^2 | \phi_\nu \rangle
&= \frac{1}{N} \int (\nabla \phi_\mu(\mathbf{r})^*) (\nabla\phi_nu(\mathbf{r})) d\mathbf{r} \\
&= \sum_{\mathbf{r}\in \Omega} w (\nabla \phi_\mu(\mathbf{r})^*) (\nabla\phi_nu(\mathbf{r}))
\end{align}

In [10]:
def get_kinetic1(cell):
    from pyscf.pbc.dft.numint import eval_ao
    grids = get_grids(cell.a, cell.mesh)
    aoR, ao_dx, ao_dy, ao_dz = eval_ao(cell, grids, deriv=1)
    w = get_weight(cell)

    t = .5 * w * (np.einsum('xi,xj->ij', ao_dx.conj(), ao_dx) +
                  np.einsum('xi,xj->ij', ao_dy.conj(), ao_dy) +
                  np.einsum('xi,xj->ij', ao_dz.conj(), ao_dz))
    return t

Method 2: numerical integration in reciprocal space

In [11]:
def get_kinetic2(cell):
    aoG = get_aoG_values(cell)
    Gv = get_Gv(cell)
    w = 1./cell.vol
    t = .5 * w * ________
    return t

def get_kinetic_ref(cell):
    return cell.pbc_intor('int1e_kin')
get_kinetic = get_kinetic_ref

if TEST:
    print(abs(get_kinetic1(cell) - get_kinetic2(cell)).max())
    print(abs(get_kinetic1(cell) - get_kinetic_ref(cell)).max())

### Nuclear attraction integrals
\begin{align}
\langle \phi_\mu | V_{nuc} | \phi_\nu \rangle
&= \frac{1}{N} \sum_{LL'L''} \int \mu^*(\mathbf{r}-\mathbf{L}) \nu(\mathbf{r}-\mathbf{L}') \frac{Z_A}{|\mathbf{r} - \mathbf{R}_A - \mathbf{L''}|} d\mathbf{r} \\
&= \sum_{LL'} \int \mu^*(\mathbf{r}-\mathbf{L}) \nu(\mathbf{r}-\mathbf{L}') \frac{Z_A}{|\mathbf{r} - \mathbf{R}_A|} d\mathbf{r} \\
&=\sum_{\mathbf{G}\neq 0} w_{\mathbf{G}} \rho_{\mu\nu}(-\mathbf{G}) V_{nuc}(\mathbf{G})
\end{align}

\begin{gather}
\rho_{\mu\nu}(\mathbf{G}) = \sum_{LL'}\int e^{-i \mathbf{G}\cdot \mathbf{r}} \mu(\mathbf{r}-\mathbf{L})^* \nu(\mathbf{r}-\mathbf{L'}) d\mathbf{r}
\\
w_{\mathbf{G}} = \frac{1}{\Omega}
\\
\rho_{nuc}(\mathbf{G}) = \sum_A \int e^{-i \mathbf{G}\cdot\mathbf{r}} Z_A\delta(\mathbf{r} - \mathbf{R}_A) d\mathbf{r}
= \sum_A Z_A e^{-i \mathbf{G}\cdot\mathbf{R}_A}
\\
V_{nuc}(\mathbf{G}) = \frac{4\pi}{G^2} \rho_{nuc}(\mathbf{G})
\end{gather}

In [12]:
from pyscf.pbc.tools import fft, ifft
def get_nuc(cell):
    R = cell.atom_coords()
    Gv = cell.get_Gv()
    SI = np.exp(-1j*np.einsum('zs,xs->zx', R, Gv))
    rho_nuc_G = -np.einsum('z,zx->x', cell.atom_charges(), SI)

    aoR = get_ao_values(cell)
    nao = aoR.shape[1]
    pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    pair_ao_G_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                           cell.mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    v = w * np.einsum('x,x,xij->ij', rho_nuc_G, coulG, pair_ao_G_space)
    return v

def get_nuc_ref(cell):
    from pyscf.pbc import df
    return df.FFTDF(cell).get_nuc()

if TEST:
    print abs(get_nuc(cell) - get_nuc_ref(cell)).max()


In [13]:
def get_hcore(cell):
    t = get_kinetic(cell)
    v = get_nuc(cell)
    return t + v

### Electron repulsion integrals (ERI)
\begin{align}
(ij|kl)
&= \int \phi_i(\mathbf{r})^* \phi_j(\mathbf{r}) \frac{1}{|\mathbf{r}-\mathbf{r'}|} \phi_k(\mathbf{r'})^* \phi_l(\mathbf{r'}) d\mathbf{r}d\mathbf{r'} \\
&=\sum_{\mathbf{G}\neq 0} w_G \frac{4\pi}{G^2} \rho_{ij}(\mathbf{G}) \rho_{kl}(-\mathbf{G})
\end{align}
where
\begin{equation}
\rho_{ij} (\mathbf{G}) = \int e^{-i \mathbf{G}\cdot\mathbf{r}} \phi_i^*(\mathbf{r}) \phi_j(\mathbf{r}) d\mathbf{r}
\end{equation}

In [14]:
def get_eri(cell):
    Gv = cell.get_Gv()

    aoR = get_ao_values(cell)
    nao = aoR.shape[1]

    pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    weight = cell.vol / np.prod(cell.mesh)
    pair_ao_G_space = fft(pair_ao_real_space.reshape(-1,nao**2).T,
                          cell.mesh).T.reshape(-1,nao,nao) * weight
    pair_ao_G_inv_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                               cell.mesh).T.reshape(-1,nao,nao) * cell.vol

    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    eri = ___________________________
    return eri

def get_eri_ref(cell):
    from pyscf.pbc import df
    nao = cell.nao_nr()
    eri = df.FFTDF(cell).get_eri(compact=False).reshape(nao,nao,nao,nao)
    return eri

if TEST:
    eri = get_eri(cell)
    print(abs(eri - get_eri_ref(cell)).max())

get_eri = get_eri_ref

In [15]:
get_eri = get_eri_ref
def get_jk(eri, dm):
    j = np.einsum('ijkl,ji->kl', eri, dm)
    k = np.einsum('ijkl,jk->il', eri, dm)
    return j, k

def get_vhf(cell, dm, eri):
    j, k = get_jk(eri, dm)
    E_coul = .5 * np.einsum('ij,ji', j-.5*k, dm)
    return j - .5 * k, E_coul


### Numerical integration for exchange-correlation functional
The total XC energy is a functional of electron density $\rho$
\begin{equation}
E_{xc} = \int e_{xc}[\rho]\, \rho(\mathbf{r}) d\mathbf{r}
\end{equation}
$e_{xc}$ is the XC energy per particle, which can be evaluated by the function pyscf.dft.libxc.eval_xc.

In the DFT method, one also needs to calculate the XC potential $V_{xc}(\mathbf{r})$:
\begin{equation}
V_{xc}(\mathbf{r}) = \frac{\delta E_{xc}}{\delta \rho} = e_{xc}[\rho] + \frac{\delta e_{xc}}{\delta \rho} \rho(\mathbf{r})
\end{equation}

and the matrix elements on top of that
\begin{align}
\langle \phi_\mu |V_{xc} |\phi_\nu\rangle
&= \int \phi_\mu(\mathbf{r})^* V_{xc}(\mathbf{r}) \phi_\mu(\mathbf{r}) d\mathbf{r} \\
&= \sum_{\mathbf{r}\in\Omega} w V_{xc}(\mathbf{r}) \rho_{\mu\nu}(\mathbf{r})
\end{align}
\begin{equation}
\rho_{\mu\nu}(\mathbf{r}) = \phi_\mu(\mathbf{r})^* \phi_\nu(\mathbf{r})
\end{equation}

When calling pyscf.dft.libxc.eval_xc function, if deriv=1 is specified, $e_{xc}$ and $V_{xc}$ can be evaluated simultaneously. (Note 
$V_{xc}$ includes two terms as shown above, not just the derivative of $e_{xc}$.)

In [16]:
def get_vxc(cell, dm, xc):
    from pyscf.dft import libxc
    aoR = get_ao_values(cell)
    pair_ao = np.einsum('xi,xj->xij', aoR.conj(), aoR)
    rho = np.einsum('xij,ji->x', pair_ao, dm)
    e_xc, v_xc = libxc.eval_xc(xc, rho, deriv=1)[:2]
    v_xc = v_xc[0]
    w = get_weight(cell)
    vxc = w * np.einsum('xij,x->ij', pair_ao, v_xc)
    E_xc = w * np.einsum('x,x', rho, e_xc)
    return vxc, E_xc

def get_vxc_ref(cell, dm, xc):
    from pyscf.pbc import dft
    mf = dft.RKS(cell)
    E_xc, vxc = mf._numint.nr_rks(cell, mf.grids, 'lda,vwn', dm)[1:]
    return vxc, E_xc

if TEST:
    dm = cell.pbc_intor('int1e_ovlp')
    vxc, E_xc = get_vxc(cell, dm, 'lda,vwn')
    vxc_ref, E_xc_ref = get_vxc_ref(cell, dm, 'lda,vwn')
    print(abs(E_xc_ref - E_xc).max())
    print(abs(vxc_ref - vxc).max())

get_vxc = get_vxc_ref

In [17]:
import scipy.linalg
def run_hf(cell):
    nelectron = cell.nelectron
    nocc = nelectron // 2

    hcore = get_hcore(cell)
    s = get_ovlp(cell)
    eri = get_eri(cell)
    dm = np.zeros_like(s)
    vhf, E_coul = get_vhf(cell, dm, eri)

    E_ewald = cell.ewald()
    E = E_ewald
    dE = 1e99

    cycle = 0
    while dE > 1e-4:
        cycle += 1
        Elast = E

        fock = hcore + vhf
        e, c = scipy.linalg.eigh(fock, s)
        dm = np.einsum('pi,qi->pq', c[:,:nocc], c[:,:nocc].conj()) * 2
        vhf, E_coul = get_vhf(cell, dm, eri)

        E_elec = np.einsum('ij,ji', hcore, dm) + E_coul
        E = E_ewald + E_elec
        dE = abs(E - Elast)
        print('SCF cycle', cycle, 'E(HF) =', E, 'dE =', dE)
    return E, c

if TEST:
    E, c = run_hf(cell)
    
    # verify it by customizing pyscf SCF object
    from pyscf.pbc import scf
    mf = scf.RHF(cell)
    mf.get_hcore = lambda *args: get_hcore(cell)
    mf.get_ovlp = lambda *args: get_ovlp(cell)
    mf._eri = get_eri(cell)
    nao = cell.nao
    mf.exxdiv = None  # Skip HF-X treatment
    mf.kernel()
    print(abs(E - mf.e_tot))

In [18]:
def ao2mo(cell, mos):
    eri = get_eri(cell)
    mo_i, mo_j, mo_k, mo_l = mos
    eri_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', eri, mo_i.conj(), mo_j, mo_k.conj(), mo_l)
    return eri_mo

## K-point sampling
### PBC basis function with k-point label
\begin{equation}
\phi^{\mathbf{k}}(\mathbf{r}) = \frac{1}{\sqrt{N}}\sum_{\mathbf{L}} e^{i\mathbf{k}\cdot\mathbf{L}} \mu(\mathbf{r}-\mathbf{L})
\end{equation}

In [19]:
def get_ao_values(cell, kpt):
    cell = cell.copy()
    grids = get_grids(cell.a, cell.mesh)
    Ls = get_lattice_Ls(cell, cell.nimgs)
    atom_ref = cell.atom_coords()

    ao = 0
    for L in Ls:
        cell.atom = [['He', atom_ref[0] + L]]
        cell.build()
        phase = np.exp(1j * np.dot(kpt, L))
        ao += cell.eval_gto('GTOval', grids) * phase
    return ao

def get_ao_values_ref(cell, kpt):
    grids = get_grids(cell.a, cell.mesh)
    ao = cell.pbc_eval_gto('GTOval', grids, kpt=kpt)
    return np.array(ao)

if TEST:
    kpt = np.random.rand(3)
    print(abs(get_ao_values(cell, kpt)
              - get_ao_values_ref(cell, kpt)).max())

get_ao_values = get_ao_values_ref

### Fourier transform with k point

\begin{equation}
\phi^\mathbf{k}(\tilde{\mathbf{G}}) = \int_\Omega e^{-i\tilde{\mathbf{G}}\cdot\mathbf{r}} \phi^\mathbf{k}(\mathbf{r}) d\mathbf{r}
\end{equation}

Due to the periodicity
\begin{equation}
\phi^\mathbf{k}(\mathbf{r}) = \phi^\mathbf{k}(\mathbf{r} + \mathbf{L})
\end{equation}
the Fourier transform will be zero for most of the planewaves unless
\begin{equation}
\tilde{\mathbf{G}} = \mathbf{G} + \mathbf{k}
\end{equation}
where the "Gv" vectors $\mathbf{G}$ are the discrete FT sample frequencies.  The DFT can be called as

\begin{equation}
\phi^\mathbf{k}(\mathbf{G} + \mathbf{k}) = \int_\Omega e^{-i\mathbf{G}\cdot\mathbf{r}} \Big[ e^{-i\mathbf{k}\cdot\mathbf{r}} \phi^\mathbf{k}(\mathbf{r}) \Big]d\mathbf{r} 
\end{equation}

In [20]:
def get_aoG_values(cell, kpt):
    aoR = get_ao_values(cell, kpt)
    grids = cell.get_uniform_grids()
    phase = ______
    nao = cell.nao_nr()
    aoG = []
    for i in range(nao):
        aoG.append(_____ * fft(np.einsum('x,x->x', phase, aoR[:,i]), cell.mesh))
    aoG = np.array(aoG).T
    return aoG

def get_aoG_values_ref(cell, kpt):
    from pyscf.pbc import df
    return df.ft_ao.ft_ao(cell, cell.get_Gv(), kpt=kpt)

if TEST:
    kpt = np.random.rand(3)
    print(abs(get_aoG_values(cell, kpt) -
              get_aoG_values_ref(cell, kpt))).max()

get_aoG_values = get_aoG_values_ref

### Overlap integrals with k point
Method 1:
\begin{equation}
\langle \phi^\mathbf{k}_\mu | \phi^\mathbf{k'}_\nu\rangle
= \frac{1}{N} \int (\phi^\mathbf{k}_\mu(\mathbf{r}))^* \phi^\mathbf{k'}_\nu(\mathbf{r}) d\mathbf{r}
\rightarrow \sum_{\mathbf{r}\in \Omega} w \phi^\mathbf{k}_\mu(\mathbf{r}))^* \phi^\mathbf{k'}_\nu(\mathbf{r})
\end{equation}

Method 2:
\begin{align}
\langle \phi^\mathbf{k}_\mu | \phi^\mathbf{k'}_\nu \rangle
&= \frac{1}{N}\sum_\mathbf{LL'}\int [e^{i\mathbf{k}\cdot\mathbf{L}}\mu(\mathbf{r}-\mathbf{L})]^* e^{i\mathbf{k'}\cdot\mathbf{L'}}\nu(\mathbf{r} - \mathbf{L}') d\mathbf{r} \\
&= \frac{1}{N}\sum_\mathbf{LL'}\int [e^{i\mathbf{k}\cdot\mathbf{L}}\mu(\mathbf{r})]^* e^{i\mathbf{k'}\cdot(\mathbf{L'}-\mathbf{L}) + i\mathbf{k'}\cdot\mathbf{L}} \nu(\mathbf{r} + \mathbf{L} - \mathbf{L}') d\mathbf{r} \\
&= \frac{1}{N}\sum_\mathbf{L} e^{i(\mathbf{k'}-\mathbf{k})\cdot\mathbf{L}}
\sum_\mathbf{L'}\int e^{i\mathbf{k'}\cdot\mathbf{L'}}\mu(\mathbf{r})^* \nu(\mathbf{r} + \mathbf{L'}) d\mathbf{r} \\
&=
\begin{cases}
\int_{-\infty}^{\infty} e^{i\mathbf{k'}\cdot\mathbf{L'}}\mu(\mathbf{r})^* \nu(\mathbf{r} + \mathbf{L'}) d\mathbf{r} & \mathbf{k} = \mathbf{k'} \\
0 & \text{otherwise}
\end{cases}
\end{align}

Why doesn't method 1 equal to method 2?

In [21]:
def get_ovlp1(cell, kpt1, kpt2):
    w = get_weight(cell)
    ao_k1 = get_ao_values(cell, kpt1)
    ao_k2 = get_ao_values(cell, kpt2)
    s = w * np.einsum('xi,xj->ij', ao_k1.conj(), ao_k2)
    return np.array(s)

if TEST:
    kpt1 = np.random.rand(3)
    kpt2 = np.random.rand(3)
    print(get_ovlp1(cell, kpt1, kpt2).round(2))


In [22]:
def get_ovlp1(cell, kpts):
    w = get_weight(cell)
    s = []
    for k, kpt in enumerate(kpts):
        ao = get_ao_values(cell, kpt)
        s.append(w * np.einsum('xi,xj->ij', ao.conj(), ao))
    return np.array(s)

def get_ovlp2(cell, kpts):
    from pyscf import gto as mole_gto
    cellL = cell.copy()
    grids = get_grids(cell.a, cell.mesh)
    Ls = get_lattice_Ls(cell, cell.nimgs)
    atom_ref = cell.atom_coords()

    s_k = []
    for k, kpt in enumerate(kpts):
        s = 0
        for L in Ls:
            cellL.atom = [['He', atom_ref[0] + L]]
            cellL.build()
            phase = np.exp(1j * np.dot(kpt, L))
            s += phase * mole_gto.intor_cross('int1e_ovlp', cell, cellL)
        s_k.append(s)
    return np.array(s_k)

def get_ovlp_ref(cell, kpts):
    return cell.pbc_intor('int1e_ovlp', kpts=kpts)
get_ovlp = get_ovlp_ref

if TEST:
    kpts = cell.make_kpts([2,1,1])
    print(abs(get_ovlp1(cell, kpts) - get_ovlp_ref(cell, kpts)).max())
    print(abs(get_ovlp2(cell, kpts) - get_ovlp_ref(cell, kpts)).max())


### Kinetic integrals with k point

In [23]:
get_aoG_values = get_aoG_values_ref
def get_kinetic1(cell, kpts):
    from pyscf.pbc.dft.numint import eval_ao
    grids = get_grids(cell.a, cell.mesh)
    w = get_weight(cell)
    t_k = []
    for k, kpt in enumerate(kpts):
        aoR, ao_dx, ao_dy, ao_dz = eval_ao(cell, grids, deriv=1, kpt=kpt)
        t = .5 * w * (np.einsum('xi,xj->ij', ao_dx.conj(), ao_dx) +
                      np.einsum('xi,xj->ij', ao_dy.conj(), ao_dy) +
                      np.einsum('xi,xj->ij', ao_dz.conj(), ao_dz))
        t_k.append(t)
    return np.array(t_k)

def get_kinetic2(cell, kpts):
    t = []
    w = 1./cell.vol
    for k, kpt in enumerate(kpts):
        aoG = get_aoG_values(cell, kpt)
        t.append(.5 * w * ______)
    return np.array(t)

def get_kinetic_ref(cell, kpts):
    return cell.pbc_intor('int1e_kin', kpts=kpts)
get_kinetic = get_kinetic_ref

if TEST:
    kpts = cell.make_kpts([2,1,1])
    print(abs(get_kinetic1(cell, kpts) -
              get_kinetic_ref(cell, kpts)).max())
    print(abs(get_kinetic2(cell, kpts) -
              get_kinetic_ref(cell, kpts)).max())


### Nuclear attraction integrals with k point

In [24]:
def get_nuc(cell, kpts):
    R = cell.atom_coords()
    Gv = cell.get_Gv()
    SI = np.exp(-1j*np.einsum('zs,xs->zx', R, Gv))
    rho_nuc_G = -np.einsum('z,zx->x', cell.atom_charges(), SI)

    nao = cell.nao_nr()
    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    nuc = []
    for k, kpt in enumerate(kpts):
        aoR = get_ao_values(cell, kpt)
        pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
        pair_ao_G_space = ifft(pair_ao_real_space.reshape(-1,nao**2).T,
                               cell.mesh).T.reshape(-1,nao,nao) * cell.vol

        nuc.append(___________)
    return np.array(nuc)

def get_nuc_ref(cell, kpts):
    from pyscf.pbc import df
    return np.array(df.FFTDF(cell, kpts).get_nuc(kpts))

if TEST:
    kpts = cell.make_kpts([2,1,1])
    print(abs(get_nuc_ref(cell, kpts)) - get_nuc(cell, kpts).max())
    
get_nuc = get_nuc_ref

In [25]:
def get_hcore(cell, kpts):
    t = get_kinetic(cell, kpts)
    v = get_nuc(cell, kpts)
    return t + v

### Coulomb integrals

In [26]:
def get_j(cell, dm, kpts1, kpts2):
    ngrids = np.prod(cell.mesh)
    rho = np.zeros(ngrids, dtype=np.complex)
    for k, kpt in enumerate(kpts1):
        aoR = get_ao_values(cell, kpt)
        pair_ao_real_space = np.einsum('xi,xj->xij', aoR.conj(), aoR)
        rho += np.einsum('xij,ji->x', pair_ao_real_space, dm[k])
    rho *= 1./len(kpts1)

    rhoG = fft(rho, cell.mesh) * cell.vol / np.prod(cell.mesh)

    w = 1./cell.vol
    nao = cell.nao_nr()
    Gv = cell.get_Gv()
    G2 = np.einsum('xs,xs->x', Gv, Gv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    vj_kpts = []
    for k, kpt in enumerate(kpts2):
        vj = ________________
        vj_kpts.append(vj)
    vj_kpts = np.array(vj_kpts)
    return vj_kpts

def get_j_ref(cell, dm, kpts1, kpts2):
    from pyscf.pbc import scf
    return scf.KRHF(cell, kpts1).get_j(cell, dm, kpts=kpts1, kpts_band=kpts2)

if TEST:
    kpts = cell.make_kpts([2,1,1])
    kpts2 = cell.make_kpts([1,3,1])
    dm = cell.pbc_intor('int1e_ovlp', kpts=kpts)
    print(abs(get_j(cell, dm, kpts, kpts2)
              - get_j_ref(cell, dm, kpts, kpts2)).max()).max()
    
get_j = get_j_ref


### Integrals for XC functional

In [27]:
def get_vxc(cell, dm, xc, kpts1, kpts2):
    from pyscf.dft import libxc
    ngrids = np.prod(cell.mesh)
    rho = np.zeros(ngrids, dtype=np.complex)
    for k, kpt in enumerate(kpts1):
        aoR = get_ao_values(cell, kpt)
        pair_ao = np.einsum('xi,xj->xij', aoR.conj(), aoR)
        rho += np.einsum('xij,ji->x', pair_ao, dm[k])
    rho *= 1./len(kpts1)
    rho = rho.real

    e_xc, v_xc = libxc.eval_xc(xc, rho, deriv=1)[:2]
    v_xc = v_xc[0]
    w = get_weight(cell)
    E_xc = w * np.einsum('x,x', rho, e_xc)
    vxc = []
    for k, kpt in enumerate(kpts2):
        vxc.append(___________)
    return np.array(vxc), E_xc

def get_vxc_ref(cell, dm, xc, kpts1, kpts2):
    from pyscf.pbc import dft
    mf = dft.KRKS(cell, kpts1)
    E_xc, vxc = mf._numint.nr_rks(cell, mf.grids, 'lda,vwn', dm, kpts=kpts1,
                                  kpts_band=kpts2)[1:]
    return vxc, E_xc

if TEST:
    kpts2 = cell.make_kpts([1,3,1])
    dm = cell.pbc_intor('int1e_ovlp', kpts=kpts)
    vxc, E_xc = get_vxc(cell, dm, 'lda,vwn', kpts, kpts2)
    vxc_ref, E_xc_ref = get_vxc_ref(cell, dm, 'lda,vwn', kpts, kpts2)
    print(abs(vxc-vxc_ref).max())
    print(abs(E_xc-E_xc_ref).max())

get_vxc = get_vxc_ref

### Band structure

In [32]:
get_nuc = get_nuc_ref
get_j = get_j_ref
get_vxc = get_vxc_ref
def get_bands(cell, mo_kpts, kpts, kpts_band):
    nelectron = cell.nelectron
    nocc = nelectron // 2
    dm = [np.einsum('pi,qi->pq', c[:,:nocc], c[:,:nocc].conj()) * 2
          for c in mo_kpts]

    hcore = (np.array(get_kinetic(cell, kpts_band)) +
             np.array(get_nuc(cell, kpts_band)))
    vj = get_j(cell, dm, kpts, kpts_band)
    vxc = get_vxc(cell, dm, 'lda,vwn', kpts, kpts_band)[0]
    fock = hcore + vj + vxc
    s = get_ovlp(cell, kpts_band)

    bands = []
    for k, kpt in enumerate(kpts_band):
        e, c = scipy.linalg.eigh(fock[k], s[k])
        bands.append(e)
    return np.array(bands)

def special_points(cell):
    points = (('G', [0, 0, 0]),
              ('A', [.5, .5, 0]),
              ('C', [0, .5, .5]),
              ('D', [.5, 0, .5]),
              ('E', [.5, .5, .5]),
              ('X', [0, .5, 0]),
              ('Y', [0, 0, .5]),
              ('Z', [.5, 0, 0]))
    kpts_band = np.einsum('is,sx->ix',
                          [x[1] for x in points],
                          cell.reciprocal_vectors())
    return points, kpts_band

if TEST:
    gamma = np.zeros((1,3))
    from pyscf.pbc import dft
    mf = dft.KRKS(cell, gamma).run()

    points, kpts_band = special_points(cell)
    bands = get_bands(cell, mf.mo_coeff, gamma, kpts_band)
    
    import matplotlib.pyplot as plt
    fig = plt.figure()
    af1 = fig.add_subplot(111)
    nbands = cell.nao_nr()
    kpath = np.arange(len(points))
    for i in range(nbands):
        af1.plot(kpath, bands[:,i], '-', color='k')
    af1.set_xlim(0, 7)
    af1.set_xticks(kpath)
    af1.set_xticklabels([x[0] for x in points])
    plt.show()
    

In [29]:
import scipy.linalg
def run_hf(cell, kpts):
    nelectron = cell.nelectron
    nocc = nelectron // 2
    nkpts = len(kpts)

    hcore = get_hcore(cell, kpts)
    s = get_ovlp(cell, kpts)
    dm = np.zeros_like(s)
    vxc, Exc = get_vxc(cell, dm, 'lda,vwn', kpts, kpts)
    vj = get_j(cell, dm, kpts, kpts)

    E_ewald = cell.ewald()
    E = E_ewald
    dE = 1e99

    cycle = 0
    while dE > 1e-4:
        cycle += 1
        Elast = E

        fock = hcore + vj + vxc
        e_k = []
        c_k = []
        for k in range(nkpts):
            e, c = scipy.linalg.eigh(fock[k], s[k])
            e_k.append(e)
            c_k.append(c)
        e_k = np.array(e_k)
        c_k = np.array(c_k)
        fermi = np.sort(e_k.ravel())[nkpts*nocc-1]
        dm = __________
        
        vxc, Exc = get_vxc(cell, dm, 'lda,vwn', kpts, kpts)
        vj = get_j(cell, dm, kpts, kpts)
        
        E_coul = .5 * np.einsum('kij,kji', vj, dm) / nkpts
        E_elec = np.einsum('kij,kji', hcore, dm) / nkpts + E_coul + Exc
        E = E_ewald + E_elec
        E = E.real
        dE = abs(E - Elast)
        print('SCF cycle', cycle, 'E(HF) =', E, 'dE =', dE)
    return E, c_k

if TEST:
    kpts = cell.make_kpts([2,2,2])
    E, c_k = run_hf(cell, kpts)

    from pyscf.pbc import dft
    mf = dft.KRKS(cell, kpts)
    mf.xc = 'lda,vwn'
    mf.kernel()
    print(abs(E - mf.e_tot))

### ERIs with k points
\begin{align}
(ij|kl)
&=\sum_{\mathbf{G}\neq 0} w_G \frac{4\pi}{|\mathbf{G} + \mathbf{k}_j - \mathbf{k}_i|^2} \rho_{ij}(\mathbf{G} + \mathbf{k}_j - \mathbf{k}_i) \rho_{kl}(-\mathbf{G} - \mathbf{k}_j + \mathbf{k}_i)
\end{align}

\begin{equation}
\rho_{ij} (\mathbf{G} + \mathbf{k}) = \int e^{-i (\mathbf{G}+\mathbf{k})\cdot\mathbf{r}} \phi^{\mathbf{k}_i}_i(\mathbf{r})^* \phi^{\mathbf{k}_j}_j(\mathbf{r}) d\mathbf{r}
\end{equation}

In [31]:
def ao2mo(cell, mos, kpts):
    # momont conservation symmetry
    dk = kpts[1] - kpts[0] + kpts[3] - kpts[2]
    if abs(dk).max() > 1e-8:
        raise RuntimeError('moment not conserved')

    mo_i, mo_j, mo_k, mo_l = mos

    mo_i = np.einsum('xp,pi->xi', get_ao_values(cell, kpts[0]), mo_i)
    mo_j = np.einsum('xp,pi->xi', get_ao_values(cell, kpts[1]), mo_j)
    mo_k = np.einsum('xp,pi->xi', get_ao_values(cell, kpts[2]), mo_k)
    mo_l = np.einsum('xp,pi->xi', get_ao_values(cell, kpts[3]), mo_l)

    ij_real_space = np.einsum('xi,xj->xij', mo_i.conj(), mo_j)
    ij_G_space = fft(___________)
    kl_G_space = ifft(____________)
    
    Gv = cell.get_Gv()
    kGv = Gv + kpts[1] - kpts[0]
    w = 1./cell.vol
    G2 = np.einsum('xs,xs->x', kGv, kGv)
    coulG = 4 * np.pi / G2
    coulG[G2 == 0] = 0

    eri_mo = w * np.einsum('x,xij,xkl->ijkl', coulG, ij_G_space, kl_G_space)
    return eri_mo

## Using DF module to access ERIs

In [ ]:
from pyscf.pbc import df
kpts = cell.make_kpts([2,2,2])
kpt_ijkl = [kpts[0], kpts[1], kpts[1], kpts[0]]

eri_ao = df.FFTDF(cell, kpts).get_eri(kpt_ijkl)
eri_mo = df.FFTDF(cell, kpts).get_eri(mos, kpt_ijkl)

eri_ao = df.DF(cell, kpts).get_eri(kpt_ijkl)
eri_mo = df.DF(cell, kpts).get_eri(mos, kpt_ijkl)

## Test system for k-point to supercell gamma point transformation

In [ ]:
if 0:
    from pyscf.pbc import gto, scf, dft, tools
    cell = gto.Cell()
    cell.atom = '''
    H 0.  0.  0.
    H 0.5 0.3 0.4
    '''

    cell.basis = [[0, (1, 1)], [1, (.5, 1)]]
    cell.a = np.eye(3) * 3.
    cell.unit = 'B'
    cell.build()

    kmesh = [2, 1, 1]
    kpts = cell.make_kpts(kmesh)
    
    kmf = scf.KRHF(cell, kpts).run()
    mo_coeff_gamma = k2gamma(cell, kpts, kmf.mo_coeff)
    
    scell = tools.super_cell(cell, kmesh)
    smf = scf.RHF(scell).run()
    
    #TODO: comparing smf.mo_coeff and mo_coeff_gamma
    